In [31]:
import pandas as pd
import numpy as np
import torch
import re
import logging
from datasets import Dataset
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from transformers import RobertaTokenizer
from transformers import RobertaForSequenceClassification, Trainer, TrainingArguments

In [22]:
cleaned_data = pd.read_pickle('../notebooks/cleaned_data.pkl')
cleaned_data

,review,sentiment,cleaned_review
0,One of the other reviewers has mentioned that ...,positive,one reviewer mentioned watching oz episode hoo...
1,A wonderful little production. <br /><br />The...,positive,wonderful little production filming technique ...
2,I thought this was a wonderful way to spend ti...,positive,thought wonderful way spend time hot summer we...
3,Basically there's a family where a little boy ...,negative,basically family little boy jake think zombie ...
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive,petter mattei love time money visually stunnin...
...,...,...,...
49995,I thought this movie did a down right good job...,positive,thought movie right good job creative original...
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative,bad plot bad dialogue bad acting idiotic direc...
49997,I am a Catholic taught in parochial elementary...,negative,catholic taught parochial elementary school nu...
49998,I'm going to have to disagree with the previou...,negative,going disagree previous comment side maltin on...


In [23]:
class TextPreprocessor(BaseEstimator, TransformerMixin):
    def __init__(self, log_every=1000):
        self.log_every = log_every

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        return np.array([
            self._preprocess(text, idx) for idx, text in enumerate(X)
        ])

    def _preprocess(self, text, idx=None):
        if idx is not None and self.log_every and idx % self.log_every == 0:
            logging.info(f"Preprocessing row {idx}")

        # Remove HTML tags
        text = re.sub(r"<.*?>", " ", text)
        # Lowercase only (keep punctuation)
        text = text.lower()
        return text

In [24]:
preprocessor = TextPreprocessor()
review_copy = cleaned_data['review'].copy()
cleaned_reviews = preprocessor.transform(review_copy)

cleaned_data['cleaned_review_roberta'] = cleaned_reviews
cleaned_data

,review,sentiment,cleaned_review,cleaned_review_roberta
0,One of the other reviewers has mentioned that ...,positive,one reviewer mentioned watching oz episode hoo...,one of the other reviewers has mentioned that ...
1,A wonderful little production. <br /><br />The...,positive,wonderful little production filming technique ...,a wonderful little production. the filming t...
2,I thought this was a wonderful way to spend ti...,positive,thought wonderful way spend time hot summer we...,i thought this was a wonderful way to spend ti...
3,Basically there's a family where a little boy ...,negative,basically family little boy jake think zombie ...,basically there's a family where a little boy ...
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive,petter mattei love time money visually stunnin...,"petter mattei's ""love in the time of money"" is..."
...,...,...,...,...
49995,I thought this movie did a down right good job...,positive,thought movie right good job creative original...,i thought this movie did a down right good job...
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative,bad plot bad dialogue bad acting idiotic direc...,"bad plot, bad dialogue, bad acting, idiotic di..."
49997,I am a Catholic taught in parochial elementary...,negative,catholic taught parochial elementary school nu...,i am a catholic taught in parochial elementary...
49998,I'm going to have to disagree with the previou...,negative,going disagree previous comment side maltin on...,i'm going to have to disagree with the previou...


In [25]:
cleaned_data['label'] = cleaned_data['sentiment'].map({'positive': 1, 'negative': 0})
cleaned_data

,review,sentiment,cleaned_review,cleaned_review_roberta,label
0,One of the other reviewers has mentioned that ...,positive,one reviewer mentioned watching oz episode hoo...,one of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,positive,wonderful little production filming technique ...,a wonderful little production. the filming t...,1
2,I thought this was a wonderful way to spend ti...,positive,thought wonderful way spend time hot summer we...,i thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,negative,basically family little boy jake think zombie ...,basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive,petter mattei love time money visually stunnin...,"petter mattei's ""love in the time of money"" is...",1
...,...,...,...,...,...
49995,I thought this movie did a down right good job...,positive,thought movie right good job creative original...,i thought this movie did a down right good job...,1
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative,bad plot bad dialogue bad acting idiotic direc...,"bad plot, bad dialogue, bad acting, idiotic di...",0
49997,I am a Catholic taught in parochial elementary...,negative,catholic taught parochial elementary school nu...,i am a catholic taught in parochial elementary...,0
49998,I'm going to have to disagree with the previou...,negative,going disagree previous comment side maltin on...,i'm going to have to disagree with the previou...,0


In [26]:
X_train, X_test, y_train, y_test = train_test_split(
    cleaned_data['cleaned_review_roberta'], cleaned_data['label'], test_size=0.2, random_state=42
)

train_df = pd.DataFrame({'text': X_train, 'label': y_train})
test_df = pd.DataFrame({'text': X_test, 'label': y_test})

train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

In [27]:
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

def tokenize_function(batch):
    return tokenizer(batch['text'], padding='max_length', truncation=True, max_length=128)

train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

Map:   0%|          | 0/39665 [00:00<?, ? examples/s]

Map:   0%|          | 0/9917 [00:00<?, ? examples/s]

In [28]:
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=2)

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=2,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_dir='./logs',
    logging_steps=10,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

trainer.train()

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss
1,0.429200,0.427643
2,0.260100,0.364104


TrainOutput(global_step=9918, training_loss=0.38429844370109945, metrics={'train_runtime': 18684.5745, 'train_samples_per_second': 4.246, 'train_steps_per_second': 0.531, 'total_flos': 5218150005427200.0, 'train_loss': 0.38429844370109945, 'epoch': 2.0})

In [32]:
model.save_pretrained('../models/roberta_sentiment')
tokenizer.save_pretrained('../models/roberta_sentiment')

SafetensorError: Error while serializing: IoError(Os { code: 1224, kind: Uncategorized, message: "The requested operation cannot be performed on a file with a user-mapped section open." })